# III. Identifying Spend Requirements

At the conclusion of the prior section Alice had identified an output that was destined to her, recovered the payment id, and recovered the payment amount. This section will calculate and recover additional information in the transaction that is required for Alice to be able to spend the output in a later transaction she will make. A later section will cover validation of a transaction by a miner, but for now it will be assumed that the transaction Alice found in the blockchain is valid.

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 5](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)


**It is highly recommended that the reader read chapter 5 of *Zero to Monero: Second Edition* prior to reading the following sections.**

The sections below cover:
1. extracting relevant data from transaction
2. calculating stealth address private key
3. calculating amount commitment blinding factor (y)
4. calculating key image (K~)

### Summary (Introduction)

**What we need (previousy calculated):**
* Alice's private view key: `e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b`
* Alice's private spend key: `30f5af5d3b2a206708c56bb7a2238d2006e47b10cc3ec92b240e1982ecbfc805`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* transaction public key (rG): `587a7d33f0b28ee85cfeeee0da7cab9ff5c348a648fee350099f95ccf2e3cc39`
* stealth address: `6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374`
* payment amount (b): `69300000`

**What we start with (from the transaction data):**
* committment

**What we end up with:**
* stealth address private key
* committment blinding factor (y)
* key image base 
* key image 

## 1. Extracting relevant data from the transaction

The one final piece of information Alice will need from the transaction data to spend the output is the amount committment. The purpose of this will be explained later. 

* amount commitment
    * included in the `rct_signatures \ outPk` section of the transaction

In [1]:
# read data directly from the saved transaction
import json

with open("../../transactions/inbound/txn_be71.json", "r") as file:
    txn = json.load(file)

committment = txn["rct_signatures"]["outPk"]

The `rct_signatures \ outPk` section contains one commitment per transaction output. The committments are in the same order as the transaction outputs. Previously Alice identified that the first transaction output was destined to her. Both committments are displayed for completeness, but only the first committment is usable by Alice.

In [2]:
'''
Zero to Monero: Second Edition; Appendix A
'''

# extract committments from list
committment_0 = committment[0]
committment_1 = committment[1]

print(f"Committment 0 (Alice's): {committment_0}")
print(f"Committment 1 (not Alice's): {committment_1}")
print(f"Field (outPk): {committment}")

Committment 0 (Alice's): 3f12cbc228a3c2d8110466bfa0db1aa36542f1f2ee30afc45767c8b99636c797
Committment 1 (not Alice's): 3b24a945cda8b1466984704b96d51bcbe77f84d60aa8be200d8de1e9503b7b36
Field (outPk): ['3f12cbc228a3c2d8110466bfa0db1aa36542f1f2ee30afc45767c8b99636c797', '3b24a945cda8b1466984704b96d51bcbe77f84d60aa8be200d8de1e9503b7b36']


## 2. Calculating stealth address private key

Recall that Alice only required her private **view** key to identify the stealth address that was destined to her. To calculate the stealth address's corresponding private key she will need to use her spend key. The stealth address private key is required to spend the output.

In [3]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To calculate the stealth address private key this code performs the following:

  1. Recalculates HrKv from the prior section for the first stealth address
  2. Adds Alice's private spend key to the step 1 output
'''

import nacl.bindings
import varint

from binascii import hexlify, unhexlify
from Cryptodome.Hash import keccak


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()


# define a function to add two scalars (eg: two integers)
scalar_add = nacl.bindings.crypto_core_ed25519_scalar_add


# previously calculated from Alice's mnemonic
private_spend_key = "30f5af5d3b2a206708c56bb7a2238d2006e47b10cc3ec92b240e1982ecbfc805"
private_view_key = "e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b"


# previously extracted from transaction data
rG = "587a7d33f0b28ee85cfeeee0da7cab9ff5c348a648fee350099f95ccf2e3cc39"


### begin section: this is repeated from 2_ownership_calculation for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
### end section


# previously calculated H(rKv)
stealth_address_private_key_0 = scalar_add(HrKv, unhexlify(private_spend_key))


print(f"Stealth address private key (Output 0): {hexlify(stealth_address_private_key_0).decode()}")

Stealth address private key (Output 0): 9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06


This section is added only to clarify that the stealth address private key corresponds to the stealth address. If the stealth address private key is correct then the stealth address can be calculated as (stealth_address_private_key * G). Recall that the stealth address from transaction data is: `6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374`.

In [4]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To recalculate the stealth address this code performs the following:

  1. Multiplies the stealth address private key by the generator point G
'''

# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# recalculate the stealth address as stealth_address_private_key * G
recalculated_stealth_address_0 = scalar_mult_G(stealth_address_private_key_0)


print(f"Stealth address (recalculated) (Output 0): {hexlify(recalculated_stealth_address_0).decode()}")
print(f"Stealth address (from transaction data) (Output 0): 6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374")

Stealth address (recalculated) (Output 0): 6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374
Stealth address (from transaction data) (Output 0): 6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374


For clarity the same calculations will be performed for the second transaction output. This should produce a *private key* that is invalid and that cannot be used to recover the stealth address. Recall that the second transaction output stealth address available from the transaction data is: `9cb1ec2a60e305dbb65c575c9bef4157ca1db7ae8d0a581aceb6f5c3855ad299`.

In [5]:
# define a function to add two scalars (eg: two integers)
scalar_add = nacl.bindings.crypto_core_ed25519_scalar_add


### begin section: this is repeated from 2_ownership_calculation for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(1)
HrKv = keccak_256(rKv_concat_extra_index)
### end section


# previously calculated H(rKv)
stealth_address_private_key_1 = scalar_add(HrKv, unhexlify(private_spend_key))
recalculated_stealth_address_1 = scalar_mult_G(stealth_address_private_key_1)


print(f"Stealth address private key (Output 1): {hexlify(stealth_address_private_key_1).decode()}")
print(f"Stealth address (recalculated) (Output 1): {hexlify(recalculated_stealth_address_1).decode()}")
print(f"Stealth address (from transaction data) (Output 1): 9cb1ec2a60e305dbb65c575c9bef4157ca1db7ae8d0a581aceb6f5c3855ad299")

Stealth address private key (Output 1): eb2f0a9a5524c4bd09ebc8b19aa7a9b07a6c988b9c70956224ac251bc37a4d03
Stealth address (recalculated) (Output 1): 6ac4a840b7c001562c297adc08dd1c2d8ccbd72a3562e05fda7f2a0c5cb263df
Stealth address (from transaction data) (Output 1): 9cb1ec2a60e305dbb65c575c9bef4157ca1db7ae8d0a581aceb6f5c3855ad299


## 3. Calculating amount commitment blinding factor

Alice previously decoded the payment amount that was stored in the transaction data field `ecdhInfo \ amount`. That field indicated she was sent 69300000 piconero, or 0.0000693 Monero. Decoding that field was all Alice needed to do to know how much Monero she was sent. Spending that Monero requires additional data. To spend Monero Alice will need to provide a validator enough information to confirm the transaction is valid without providing a specific amount. She will need to use data that was also used in calculating the commitment in the transaction destined to her. The outPk field contains a commitment to the amount Alice recieved, and the commitment is a curve point. When Alice spends her Monero she will need to create a new commitment, and will require data that was used in the existing commitment. The specifics of the sending process will be walked through later. This section will cover recreating the required data from the existing commitment. 

In [6]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To decode the commitment field this code performs the following:

  1. Concatenates the byte string "commitment_mask" and the value HrKv
  2. Ensures this is a valid curve scalar and stores it as "y" - the blinding factor
'''

import nacl.bindings

from binascii import hexlify, unhexlify


# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(point):
    point_padded_to_64_bytes = point + (64 - len(point)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


### begin section: this is repeated from 2_ownership_calculation for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
HrKv = pad_and_reduce(HrKv)
### end section


#  y = hash("commitment_mask" + hash(rKv + index))
y = keccak_256(b"commitment_mask" + HrKv)

# ensure y is a valid ed25519 scalar
y = pad_and_reduce(y)


print(f"Blinding factor (y): {hexlify(y).decode()}")

Blinding factor (y): 67f2cd5597666bef76bf1dd4a40d201ea4b4013848332b68bbabcbbe8bf9a408


Alice has obtained the blinding factor (y) she will need to spend the payment amount in future transactions. The following section is added for clarity to show that the commitment in the transaction was created using the blinding factor (y) and the payment amount (b). Specifically the commitment was calculated as: commitment = yG + bH. H is different curve point for which no one knows the private key. It is calculated from G.

In [7]:
'''
Zero to Monero: Second Edition; Section 5.3
'''


'''
To recreate the commitment field this code performs the following:

  1. Calculate yG by multiplying the blinding factor (y) by the generator point G
  2. Create the curve point H from the generator point G
  3. Encode the amount (b) as bytes
  4. Calculate bH by multiplying the amount (b) by the point H
  5. Add yG and bH 
'''

# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# Monero multiplies curve points by 8, create 8 in bytes for use in calculations
cofactor = int(8).to_bytes(32, byteorder="little")


# calculate yG
yG = scalar_mult_G(y)


### begin section: recreate the curve point H
# obtain the curve point G (uses G*1 = G)
G = scalar_mult_G(int(1).to_bytes(32, byteorder="little"))

# hash G to a number and directly interpret this as the y-coordinate for a curve point
H = keccak_256(G)

# multiply H by the cofactor 8 to ensure it is in the prime order subgroup
H = scalar_point_mult(cofactor, H)
### end section: recreate the curve point H


# recode payment amount as 32 bytes
b = int(69300000).to_bytes(32, byteorder="little")

# calculate bH
bH = scalar_point_mult(b, H)

# calculate commitment: yG + bH
recalculated_committment_0 = point_add(yG, bH)


print(f"Curve point yG: {hexlify(yG).decode()}")
print(f"Curve point bH: {hexlify(bH).decode()}")
print(f"Commitment (recalculated) (Output 0): {hexlify(recalculated_committment_0).decode()}")
print(f"Commitment (from transaction data) (Output 0): {committment_0}")

Curve point yG: 8940ca12e99330f303bd93690ddbcfda2995f623b31ab0129b8b91d6a4857321
Curve point bH: e47367cf908541071c082a16b68250b5710b08ac5a9b16072b216e9dbea85d31
Commitment (recalculated) (Output 0): 3f12cbc228a3c2d8110466bfa0db1aa36542f1f2ee30afc45767c8b99636c797
Commitment (from transaction data) (Output 0): 3f12cbc228a3c2d8110466bfa0db1aa36542f1f2ee30afc45767c8b99636c797


One question that may arise is, why can't the amount be recovered directly from the commitment rather than being stored again in a separate location `ecdhInfo \ amount`? Recall that the commitment is yG + bH. Alice has access to G and H (known curve points), and can calculate y and thus yG. Alice could subtract yG from the commitment to obtain bH. Given bH and H can Alice recover b? No. In this instance b is the private key for the curve point bH, and it is not possible to recover a prive key. That is why the amount is stored in both the commitment and in the amount field.

## 4. Calculate key image

The use of a key image will be explained in a later section when Alice sends her Monero to Bob. The calculations are included in this section because the key image and key image base are calculated using only data from the received transaction and Alice's keys.

In [8]:
'''
Zero to Monero: Second Edition; Section 3.4
'''


'''
To the key image this code performs the following:

  1. Calculate a keccak hash of the stealth address
  2. Apply modulus to ensure hash is within the curve order
  3. Directly convert the hashed value to a curve point
  4. Multiply the curve point by the stealth address private key
'''

import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


stealth_address = "6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374"


# hash public key, convert to integer modulus prime field, convert back to bytes
hashed_stealth_address_public_key = keccak.new(digest_bits=256).update(unhexlify(stealth_address)).digest()
q = 2**255 - 19
hash_as_int = int.from_bytes(hashed_stealth_address_public_key, byteorder="little") % q
hash_as_int = hash_as_int.to_bytes(32, byteorder="little")

# use nacl bindings to map modified hashed value to a curve point
curve_point_from_hash = nacl.bindings.crypto_core_ed25519_from_uniform(hash_as_int)

# need to review: nacl compression is resulting in a different encoding than Monero
msb = curve_point_from_hash[-1] | 0x80
modified_curve_point_from_hash = curve_point_from_hash[:-1] + bytes([msb])

# calculate key image
key_image = nacl.bindings.crypto_scalarmult_ed25519_noclamp(stealth_address_private_key_0, modified_curve_point_from_hash)

print(f"Key image base (Output 0): {hexlify(modified_curve_point_from_hash).decode()}")
print(f"Key image (Output 0): {hexlify(key_image).decode()}")

Key image base (Output 0): a05f0062aea80f577d029bb93683d3c55fa8378510e9a7350cc2a778814ee1dd
Key image (Output 0): 5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01


### Summary (Introduction)

**What we need (previousy calculated):**
* Alice's private view key: `e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b`
* Alice's private spend key: `30f5af5d3b2a206708c56bb7a2238d2006e47b10cc3ec92b240e1982ecbfc805`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* transaction public key (rG): `587a7d33f0b28ee85cfeeee0da7cab9ff5c348a648fee350099f95ccf2e3cc39`
* stealth address: `6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374`
* payment amount (b): `69300000`

**What we start with (from the transaction data):**
* amount committment: `3f12cbc228a3c2d8110466bfa0db1aa36542f1f2ee30afc45767c8b99636c797`

**What we end up with:**
* stealth address private key: `9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06`
* committment blinding factor (y): `67f2cd5597666bef76bf1dd4a40d201ea4b4013848332b68bbabcbbe8bf9a408`
* key image base: `a05f0062aea80f577d029bb93683d3c55fa8378510e9a7350cc2a778814ee1dd`
* key image (K~): `5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01`